In [10]:
import os
import cv2
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from skimage.feature import hog
from skimage import exposure

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
train_path = '/content/drive/MyDrive/KKDL/dataset_classifier/training'
val_path = '/content/drive/MyDrive/KKDL/dataset_classifier/validation'
test_path = '/content/drive/MyDrive/KKDL/dataset_classifier/testing'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)
val_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(32, 32),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=(32, 32),
    batch_size=32,
    class_mode='categorical'
)

def extract_hog_features(image_path):
    image = cv2.imread(image_path)
    resized_image = cv2.resize(image, (32, 32))  # Resize all images to a fixed size
    gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    # Apply HOG feature extraction
    fd, hog_image = hog(gray, orientations=9, pixels_per_cell=(8, 8),
                        cells_per_block=(2, 2), visualize=True, block_norm='L2-Hys')
    # Rescale HOG features for better visualization
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
    return fd

def prepare_data(directory):
    features = []
    labels = []
    for class_name in os.listdir(directory):
        class_path = os.path.join(directory, class_name)
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            features.append(extract_hog_features(image_path))
            labels.append(class_name)
    return np.array(features), np.array(labels)


train_features, train_labels = prepare_data(train_path)
val_features, val_labels = prepare_data(val_path)

# Train Decision Tree model
# model = SVC(kernel='rbf', C=100000, gamma=0.001, verbose=1)
# model = DecisionTreeClassifier(criterion="gini", max_depth=50, min_samples_leaf=5)
model = KNeighborsClassifier(n_neighbors=5)
model = RandomForestClassifier(n_estimators=950, max_depth=200, random_state=42)

model.fit(train_features, train_labels)

# Predict on validation set
val_predictions = model.predict(val_features)

# Calculate accuracy
accuracy = accuracy_score(val_labels, val_predictions)
print("Accuracy:", accuracy)
# Load an image for prediction
def predict_image(image_path, model):
    # Extract HOG features from the image
    features = extract_hog_features(image_path)
    # Reshape features to match the format expected by the model
    features = features.reshape(1, -1)  # Reshape to (1, n) where n is the number of features
    # Predict the label of the image
    prediction = model.predict(features)
    return prediction[0]

# Example usage:
new_image_path = '/content/drive/MyDrive/KKDL/dataset_classifier/testing/grasshopper/grasshopper_160.jpg'
predicted_label = predict_image(new_image_path, model)
print("Predicted label:", predicted_label)

Found 1944 images belonging to 10 classes.
Found 699 images belonging to 10 classes.
Accuracy: 0.6795422031473534
Predicted label: grasshopper
